In [37]:
import torch 
import pandas as pd 
import networkx as nx 
from itertools import combinations
from torch_geometric.data import Data 
from model import GAT

In [38]:
# Đọc dataset
file_path = r'D:\Năm 3 - HK2\Mạng xã hội\traffic-accident-analysis\data\encoded_dataset_cleaned.csv'
df = pd.read_csv(file_path)

df = df.sample(2000) # Lấy ngẫu nhiên 2000 dòng dữ liệu

print(df.columns)
print(df['damage'].value_counts())
print(df['damage'].unique())

Index(['traffic_control_device', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment',
       'roadway_surface_cond', 'road_defect', 'crash_type',
       'intersection_related_i', 'damage', 'prim_contributory_cause',
       'num_units', 'most_severe_injury', 'injuries_total', 'injuries_fatal',
       'injuries_incapacitating', 'injuries_non_incapacitating',
       'injuries_reported_not_evident', 'injuries_no_indication', 'crash_hour',
       'crash_day_of_week', 'crash_month'],
      dtype='object')
damage
2    1384
1     409
0     207
Name: count, dtype: int64
[1 2 0]


In [39]:
# Khởi tạo đồ thị
G = nx.Graph()

# Thêm nút cho mỗi vụ tai nạn (dựa trên index)
for index, row in df.iterrows():
    G.add_node(index,
               traffic_control_device=row['traffic_control_device'],
               weather_condition=row['weather_condition'],
               lighting_condition=row['lighting_condition'],
               first_crash_type=row['first_crash_type'],
               trafficway_type=row['trafficway_type'],
               alignment=row['alignment'],
               roadway_surface_cond=row['roadway_surface_cond'],
               road_defect=row['road_defect'],
               crash_type=row['crash_type'],
               intersection_related_i=row['intersection_related_i'],
               damage=row['damage'],
               prim_contributory_cause=row['prim_contributory_cause'],
               num_units=row['num_units'],
               most_severe_injury=row['most_severe_injury'],
               injuries_total=row['injuries_total'],
               injuries_fatal=row['injuries_fatal'],
               injuries_incapacitating=row['injuries_incapacitating'],
               injuries_non_incapacitating=row['injuries_non_incapacitating'],
               injuries_reported_not_evident=row['injuries_reported_not_evident'],
               injuries_no_indication=row['injuries_no_indication'],
               crash_hour=row['crash_hour'],
               crash_day_of_week=row['crash_day_of_week'],
               crash_month=row['crash_month'])

# Hàm kiểm tra điều kiện kết nối giữa hai vụ tai nạn
def is_similar(accident1, accident2):
    # Các điều kiện tương tự dựa trên các đặc trưng quan trọng
    time_diff = abs(accident1['crash_hour'] - accident2['crash_hour']) <= 1
    same_month = accident1['crash_month'] == accident2['crash_month']
    same_day_of_week = accident1['crash_day_of_week'] == accident2['crash_day_of_week']
    same_trafficway = accident1['trafficway_type'] == accident2['trafficway_type']
    same_crash_type = accident1['first_crash_type'] == accident2['first_crash_type']
    same_injury_no_indication = accident1['injuries_no_indication'] == accident2['injuries_no_indication']

    # Kết nối nếu ít nhất một điều kiện tương tự
    return (time_diff or same_month or same_day_of_week or same_trafficway or
            same_crash_type or same_injury_no_indication)

# Thêm các cạnh dựa trên tính tương đồng
for u, v in combinations(G.nodes(data=True), 2):
    if is_similar(u[1], v[1]):
        G.add_edge(u[0], v[0])

print("Đồ thị G đã được tạo với", G.number_of_nodes(), "nút và", G.number_of_edges(), "cạnh.")

Đồ thị G đã được tạo với 2000 nút và 1435958 cạnh.


In [40]:
# Định nghĩa Focal Loss
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, targets):
        ce_loss = F.cross_entropy(logits, targets, weight=self.alpha, reduction="none")
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

# Chuyển đổi từ NetworkX sang PyG Data (giả định G đã được định nghĩa)
def networkx_to_pyg(G, label_attr="damage"):
    node_mapping = {node: i for i, node in enumerate(G.nodes())}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in G.edges()], dtype=torch.long).t().contiguous()

    features = []
    labels = []
    for node, data in G.nodes(data=True):
        node_features = [data[attr] for attr in data if attr != label_attr]
        features.append(node_features)
        labels.append(data[label_attr])

    X = torch.tensor(features, dtype=torch.float)
    y = torch.tensor(labels, dtype=torch.long)
    return Data(x=X, edge_index=edge_index, y=y)

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT(in_features=22, hidden_dim=16, out_features=3, heads=8).to(device)
model.load_state_dict(torch.load('gat_model.pth', map_location=device))
model.eval()


GAT(
  (gat1): GATConv(22, 16, heads=8)
  (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (gat2): GATConv(128, 16, heads=4)
  (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (gat3): GATConv(64, 3, heads=1)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [49]:
from collections import Counter, defaultdict

def analyze_high_damage_nodes(G, label_attr='damage', high_level=1, top_k=3):
    # Bước 1: Lọc các node có predicted_damage bằng high_level
    high_damage_nodes = [n for n, data in G.nodes(data=True) if data.get(label_attr) == high_level]

    # Kiểm tra nếu không có node nào
    if not high_damage_nodes:
        print(f"No nodes found with {label_attr} = {high_level}")
        return

    # Bước 2: Thống kê các feature xuất hiện trong nhóm này
    feature_counters = defaultdict(Counter)

    for n in high_damage_nodes:
        node_data = G.nodes[n]
        for attr, value in node_data.items():
            if attr != label_attr:
                feature_counters[attr][value] += 1

    # Bước 3: In ra top K giá trị phổ biến nhất cho từng feature
    print(f"\nTop nguyên nhân thường thấy khi `{label_attr} = {high_level}`:\n")
    for attr, counter in feature_counters.items():
        print(f"- {attr}:")
        for val, freq in counter.most_common(top_k):
            print(f"   • {val}: {freq} lần")
        print()


In [50]:
analyze_high_damage_nodes(G, label_attr='damage', high_level=0)


Top nguyên nhân thường thấy khi `damage = 0`:

- traffic_control_device:
   • 16: 132 lần
   • 15: 41 lần
   • 4: 26 lần

- weather_condition:
   • 2: 150 lần
   • 7: 30 lần
   • 11: 11 lần

- lighting_condition:
   • 3: 137 lần
   • 1: 46 lần
   • 0: 10 lần

- first_crash_type:
   • 9: 55 lần
   • 10: 41 lần
   • 17: 37 lần

- trafficway_type:
   • 8: 76 lần
   • 6: 40 lần
   • 2: 39 lần

- alignment:
   • 3: 204 lần
   • 4: 2 lần
   • 0: 1 lần

- roadway_surface_cond:
   • 0: 146 lần
   • 6: 36 lần
   • 5: 14 lần

- road_defect:
   • 1: 160 lần
   • 5: 44 lần
   • 4: 1 lần

- crash_type:
   • 1: 127 lần
   • 0: 80 lần

- intersection_related_i:
   • 1: 191 lần
   • 0: 16 lần

- prim_contributory_cause:
   • 36: 63 lần
   • 18: 38 lần
   • 19: 23 lần

- num_units:
   • 2: 201 lần
   • 3: 4 lần
   • 1: 1 lần

- most_severe_injury:
   • 2: 129 lần
   • 3: 53 lần
   • 4: 14 lần

- injuries_total:
   • 0: 129 lần
   • 1: 72 lần
   • 2: 4 lần

- injuries_fatal:
   • 0: 207 lần

- injuries

In [45]:
analyze_high_damage_nodes(G, label_attr='damage', high_level=1)


Top nguyên nhân thường thấy khi `damage = 1`:

- traffic_control_device:
   • 16: 259 lần
   • 15: 78 lần
   • 4: 59 lần

- weather_condition:
   • 2: 322 lần
   • 7: 40 lần
   • 3: 17 lần

- lighting_condition:
   • 3: 269 lần
   • 1: 93 lần
   • 4: 18 lần

- first_crash_type:
   • 17: 128 lần
   • 10: 104 lần
   • 0: 57 lần

- trafficway_type:
   • 8: 182 lần
   • 6: 87 lần
   • 2: 59 lần

- alignment:
   • 3: 398 lần
   • 4: 10 lần
   • 5: 1 lần

- roadway_surface_cond:
   • 0: 302 lần
   • 6: 59 lần
   • 5: 32 lần

- road_defect:
   • 1: 332 lần
   • 5: 73 lần
   • 2: 2 lần

- crash_type:
   • 1: 328 lần
   • 0: 81 lần

- intersection_related_i:
   • 1: 386 lần
   • 0: 23 lần

- prim_contributory_cause:
   • 36: 135 lần
   • 19: 55 lần
   • 18: 53 lần

- num_units:
   • 2: 391 lần
   • 3: 11 lần
   • 1: 7 lần

- most_severe_injury:
   • 2: 347 lần
   • 3: 37 lần
   • 4: 18 lần

- injuries_total:
   • 0: 347 lần
   • 1: 51 lần
   • 2: 9 lần

- injuries_fatal:
   • 0: 409 lần

- inj

In [46]:
analyze_high_damage_nodes(G, label_attr='damage', high_level=2)


Top nguyên nhân thường thấy khi `damage = 2`:

- traffic_control_device:
   • 16: 797 lần
   • 15: 347 lần
   • 4: 188 lần

- weather_condition:
   • 2: 1099 lần
   • 7: 137 lần
   • 11: 49 lần

- lighting_condition:
   • 3: 865 lần
   • 1: 368 lần
   • 0: 48 lần

- first_crash_type:
   • 17: 478 lần
   • 0: 376 lần
   • 10: 242 lần

- trafficway_type:
   • 8: 497 lần
   • 6: 347 lần
   • 2: 211 lần

- alignment:
   • 3: 1353 lần
   • 4: 23 lần
   • 2: 4 lần

- roadway_surface_cond:
   • 0: 1045 lần
   • 6: 216 lần
   • 5: 80 lần

- road_defect:
   • 1: 1157 lần
   • 5: 209 lần
   • 2: 8 lần

- crash_type:
   • 0: 695 lần
   • 1: 689 lần

- intersection_related_i:
   • 1: 1328 lần
   • 0: 56 lần

- prim_contributory_cause:
   • 36: 388 lần
   • 18: 281 lần
   • 6: 129 lần

- num_units:
   • 2: 1207 lần
   • 3: 96 lần
   • 1: 54 lần

- most_severe_injury:
   • 2: 1009 lần
   • 3: 230 lần
   • 4: 111 lần

- injuries_total:
   • 0: 1009 lần
   • 1: 258 lần
   • 2: 73 lần

- injuries_fata